## COVID-19

### Step 2 - rework the incoming data

Data from the first step can be used in the existing visualizations.
If the data changes, then this step will be required.
Until then, this work is in progress as an exploration of other stories the data can reveal.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re
import geonamescache

from unidecode import unidecode

# PEP 484 type hints are easier with this
from typing import Tuple

## GLOBALS

TODO: Move global variables here once the dust has settled.

In [2]:
fname = 'most_recent_COVID-19.csv'

GLOBAL_FILL_STRING = 'not_provided'

gc = geonamescache.GeonamesCache()

countries = gc.get_countries()
us_states = gc.get_us_states()

In [3]:
# Country divisions - 1st level are all going to be called "states" for simplicity
us_states = gc.get_us_states()
stateID_list = list(us_states.keys())

australian_states_list = ['New South Wales', 'Queensland','South Australia','Tasmania','Victoria','Western Australia',\
                    'Australian Capital Territory','Jervis Bay Territory','Northern Territory',\
                    'Ashmore and Cartier Islands','Australian Antarctic Territory','Christmas Island',\
                    'Cocos (Keeling) Islands','Coral Sea Islands','Heard Island and McDonald Islands',\
                    'Norfolk Island']

canadian_states_list = ['British Columbia','Alberta','Saskatchewan','Manitoba','Ontario','Quebec','Nova Scotia',\
                   'New Brunswick','Prince Edward Island','Newfoundland and Labrador',\
                   'Yukon Territory','Northwest Territory', 'Nunavut Territory']

## EXPLORE

In [4]:
# Read in starting data

df = pd.read_csv(fname, parse_dates=['Date'])

In [5]:
df.dtypes

Province/State            object
Country/Region            object
Lat                      float64
Long                     float64
Date              datetime64[ns]
Confirmed                  int64
Deaths                     int64
Recovered                  int64
dtype: object

In [6]:
df[['Province/State','Country/Region']] = df[['Province/State','Country/Region']].fillna(GLOBAL_FILL_STRING)

In [7]:
df.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,not_provided,Thailand,15.0000,101.0000,2020-01-22,2,0,0
1,not_provided,Japan,36.0000,138.0000,2020-01-22,2,0,0
2,not_provided,Singapore,1.2833,103.8333,2020-01-22,0,0,0
3,not_provided,Nepal,28.1667,84.2500,2020-01-22,0,0,0
4,not_provided,Malaysia,2.5000,112.5000,2020-01-22,0,0,0


In [8]:
df['Country/Region'].value_counts()

United States     12150
China              1650
Australia           400
Canada              250
United Kingdom      150
                  ...  
Colombia             50
Mongolia             50
Lithuania            50
Qatar                50
Liechtenstein        50
Name: Country/Region, Length: 114, dtype: int64

In [9]:
def handle_special_locations(name:str, verbose=False)->bool:
    '''Catch the special locations...
        print them out if they are new
        otherwise just return True so that they are flagged
    '''
    if (name == "From Diamond Princess"):
        return True
    elif (name == GLOBAL_FILL_STRING):
        return False
    else:
        if (verbose):
            print("Unrecognized location: {}".format(name))
        return False

In [10]:
def test__handle_special_locations()->bool:
    '''Unit test'''
    b1T = handle_special_locations("From Diamond Princess")
    b2F = handle_special_locations(GLOBAL_FILL_STRING)
    b3F = handle_special_locations("That other cruise ship")
    b4F = handle_special_locations("And this cruise ship",True)
    
    if (b1T & ~(b2F | b3F | b4F)):
        print("test complete: all good")
        return True
    else:
        return False

test__handle_special_locations()


Unrecognized location: And this cruise ship
test complete: all good


True

In [11]:
def split_US_county_state(ps: str)->Tuple[str,str,bool]:
    '''Take in the string for Province/State and return 3 pieces:
        my_county : str
        my_state  : str
        my_special : bool   # For things like cruise ships
    '''
    # default values
    my_county = GLOBAL_FILL_STRING
    my_state = GLOBAL_FILL_STRING
    my_special = False
    
    if (ps == GLOBAL_FILL_STRING):
        # no Province/State info - keep the default values    
        pass
    
    elif (handle_special_locations(ps)):
        # Diamond Princess and similar
        my_county = GLOBAL_FILL_STRING
        my_state = GLOBAL_FILL_STRING
        my_special = True 
        
    else:

        uni_search = r'(?P<county>.*), (?P<state2letter>[A-Z][A-Z])($|\b)'
        # \b is a word boundary like white space      
        #print("uni_search is: .{}.".format(uni_search))

        regexp = re.compile(uni_search,flags=re.IGNORECASE)
        result = regexp.search(ps)
        #print ("   result is {}".format(result))
            
         # older data has just the state, not county, ST, so just copy over
        if (result == None):             
            stateName = ps  
            countyName = GLOBAL_FILL_STRING
        else:
            stateID = result.group('state2letter')               
            if (stateID not in stateID_list):                      # typo? or change in source data?
                print("WARNING: stateID {} not in the list of states for the United States.".format(stateID))
                stateName = us_states[stateID]['name'] 
            else:
                stateName = us_states[stateID]['name']
                                  
            # Get the county name, and make sure it includes the word "County"
            countyName = result.group('county')
            #regex_county = re.compile('county',flags=re.IGNORECASE)
            #result = regex_county.search(countyName)
            if (result == None):
                countyName = countyName + ' County'
                    
        my_state = stateName                
        my_county = countyName
    return my_county, my_state, my_special


In [12]:
def split_ProvinceState(row: pd.core.series.Series,debug=False)-> pd.core.series.Series:
    '''Split Province/State into 3 parts:  State, County, City'''
    
    ps = row['Province/State']
    if (debug):
        print("Province/State is {} which is of type {}".format(ps, type(ps)))
    
    # Rather than fill these in for every case, start with the 'generic' and over-write if they need updates
    row['State'] = GLOBAL_FILL_STRING
    row['County'] = GLOBAL_FILL_STRING
    row['isSpecialLocation'] = False   # such as cruise ships
    
    # Separate county, state
    if (row['Country/Region'] == 'United States'):                        # United States
        str1, str2, mybool = split_US_county_state(ps)
        row['County'] = str1
        row['State']  = str2 
        row['isSpecialLocation'] = mybool
        
    elif (row['Country/Region'] == 'Australia'):                          # Australia
        if (ps in australian_states_list):
            row['State'] = ps
            row['County'] = GLOBAL_FILL_STRING           
        else:    
            print("WARNING: stateID {} not in the list of states for Australia.".format(ps))
            row['isSpecialLocation'] = handle_special_locations(ps)
        
    elif (row['Country/Region'] == 'Canada'):                            # Canada
        if (ps in canadian_states_list):
            row['State'] = ps
            row['County'] = GLOBAL_FILL_STRING           
        else:    
            print("WARNING: stateID {} not in the list of states for Canada.".format(ps))
            row['isSpecialLocation'] = handle_special_locations(ps)           
            
                                                                        # TODO: France, UK, China

    else:                # many countries don't have or often don't use a state-level division
        row['State'] = GLOBAL_FILL_STRING
        row['County'] = GLOBAL_FILL_STRING
        row['isSpecialLocation'] = handle_special_locations(ps)
    
    return row

In [13]:
# df = df.apply(split_ProvinceState, axis=1)
df = df.apply(lambda row: split_ProvinceState(row,True), axis=1)

Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is British Columbia which is of type <class 'str'>
Province/State is New South Wales which is of type <class 'str'>
Province/State is Victoria which is of type <class 'str'>
Province/State is Queensland which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Provinc

Province/State is Jiangxi which is of type <class 'str'>
Province/State is Shandong which is of type <class 'str'>
Province/State is Diamond Princess which is of type <class 'str'>
Province/State is Jiangsu which is of type <class 'str'>
Province/State is Chongqing which is of type <class 'str'>
Province/State is Sichuan which is of type <class 'str'>
Province/State is Heilongjiang which is of type <class 'str'>
Province/State is UK which is of type <class 'str'>
Province/State is Denmark which is of type <class 'str'>
Province/State is Beijing which is of type <class 'str'>
Province/State is Shanghai which is of type <class 'str'>
Province/State is Hebei which is of type <class 'str'>
Province/State is Fujian which is of type <class 'str'>
Province/State is Guangxi which is of type <class 'str'>
Province/State is Shaanxi which is of type <class 'str'>
Province/State is Yunnan which is of type <class 'str'>
Province/State is Hainan which is of type <class 'str'>
Province/State is Guizh

Province/State is Fayette County, KY which is of type <class 'str'>
Province/State is Marion County, IN which is of type <class 'str'>
Province/State is Middlesex County, MA which is of type <class 'str'>
Province/State is Nassau County, NY which is of type <class 'str'>
Province/State is Ramsey County, MN which is of type <class 'str'>
Province/State is Washoe County, NV which is of type <class 'str'>
Province/State is Wayne County, PA which is of type <class 'str'>
Province/State is Yolo County, CA which is of type <class 'str'>
Province/State is Santa Clara County, CA which is of type <class 'str'>
Province/State is Clark County, NV which is of type <class 'str'>
Province/State is Fort Bend County, TX which is of type <class 'str'>
Province/State is Grant County, WA which is of type <class 'str'>
Province/State is Santa Rosa County, FL which is of type <class 'str'>
Province/State is Williamson County, TN which is of type <class 'str'>
Province/State is New York County, NY which is 

Province/State is Tasmania which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is Northern Territory which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Provin

Province/State is Gwinnett, GA which is of type <class 'str'>
Province/State is DeKalb, GA which is of type <class 'str'>
Province/State is Floyd, GA which is of type <class 'str'>
Province/State is Fayette, GA which is of type <class 'str'>
Province/State is Gregg, TX which is of type <class 'str'>
Province/State is Monmouth, NJ which is of type <class 'str'>
Province/State is Burlington, NJ which is of type <class 'str'>
Province/State is Camden, NJ which is of type <class 'str'>
Province/State is Passaic, NJ which is of type <class 'str'>
Province/State is Union, NJ which is of type <class 'str'>
Province/State is Eagle, CO which is of type <class 'str'>
Province/State is Larimer, CO which is of type <class 'str'>
Province/State is Arapahoe, CO which is of type <class 'str'>
Province/State is Gunnison, CO which is of type <class 'str'>
Province/State is Kane, IL which is of type <class 'str'>
Province/State is Monroe, PA which is of type <class 'str'>
Province/State is Philadelphia,

...

Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is Western Australia which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is Tasmania which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Provinc

Province/State is Bennington County, VT which is of type <class 'str'>
Province/State is Carver County, MN which is of type <class 'str'>
Province/State is Charlotte County, FL which is of type <class 'str'>
Province/State is Cherokee County, GA which is of type <class 'str'>
Province/State is Collin County, TX which is of type <class 'str'>
Province/State is Jefferson County, KY which is of type <class 'str'>
Province/State is Jefferson Parish, LA which is of type <class 'str'>
Province/State is Shasta County, CA which is of type <class 'str'>
Province/State is Spartanburg County, SC which is of type <class 'str'>
Province/State is Harrison County, KY which is of type <class 'str'>
Province/State is Johnson County, IA which is of type <class 'str'>
Province/State is Berkshire County, MA which is of type <class 'str'>
Province/State is Davidson County, TN which is of type <class 'str'>
Province/State is Douglas County, OR which is of type <class 'str'>
Province/State is Fresno County, 

Province/State is Hanover, VA which is of type <class 'str'>
Province/State is Lancaster, SC which is of type <class 'str'>
Province/State is Sullivan, TN which is of type <class 'str'>
Province/State is Johnson, IN which is of type <class 'str'>
Province/State is Howard, IN which is of type <class 'str'>
Province/State is St. Joseph, IN which is of type <class 'str'>
Province/State is Knox, NE which is of type <class 'str'>
Province/State is Stark, OH which is of type <class 'str'>
Province/State is Anoka, MN which is of type <class 'str'>
Province/State is Olmsted, MN which is of type <class 'str'>
Province/State is Summit, UT which is of type <class 'str'>
Province/State is Fairfield, CT which is of type <class 'str'>
Province/State is Litchfield, CT which is of type <class 'str'>
Province/State is Orleans, LA which is of type <class 'str'>
Province/State is Pennington, SD which is of type <class 'str'>
Province/State is Beadle, SD which is of type <class 'str'>
Province/State is Ch

Province/State is Hawaii which is of type <class 'str'>
Province/State is Oklahoma which is of type <class 'str'>
Province/State is Utah which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is not_provided which is of type <class 'str'>
Province/State is Kansas which is of type <class 'str'>
Province/State is Louisiana which is of type <class 'str'>
Province/State is Missouri which is of type <class 'str'>
Province/State is Vermont which is of type <class 'str'>
Province/State is Alaska which is of type <class 'str'>
Province/State is Arkansas which is of type <class 'str'>
Province/State is Delaware which is of type <class 'str'>
Province/State is Idaho which is of type <class 'str'>
Province/State is Maine which is of type <class 'str'>
Province/State is Michigan which is of type <class 'str'>
Province/State

Province/State is El Paso County, CO which is of type <class 'str'>
Province/State is Honolulu County, HI which is of type <class 'str'>
Province/State is Jackson County, OR  which is of type <class 'str'>
Province/State is Jefferson County, WA which is of type <class 'str'>
Province/State is Kershaw County, SC which is of type <class 'str'>
Province/State is Klamath County, OR which is of type <class 'str'>
Province/State is Madera County, CA which is of type <class 'str'>
Province/State is Pierce County, WA which is of type <class 'str'>
Province/State is Tulsa County, OK which is of type <class 'str'>
Province/State is Douglas County, CO which is of type <class 'str'>
Province/State is Providence County, RI which is of type <class 'str'>
Province/State is Chatham County, NC which is of type <class 'str'>
Province/State is Delaware County, PA which is of type <class 'str'>
Province/State is Douglas County, NE which is of type <class 'str'>
Province/State is Fayette County, KY which i

In [14]:
df.head()


,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,State,County,isSpecialLocation
0,not_provided,Thailand,15.0000,101.0000,2020-01-22,2,0,0,not_provided,not_provided,False
1,not_provided,Japan,36.0000,138.0000,2020-01-22,2,0,0,not_provided,not_provided,False
2,not_provided,Singapore,1.2833,103.8333,2020-01-22,0,0,0,not_provided,not_provided,False
3,not_provided,Nepal,28.1667,84.2500,2020-01-22,0,0,0,not_provided,not_provided,False
4,not_provided,Malaysia,2.5000,112.5000,2020-01-22,0,0,0,not_provided,not_provided,False


In [15]:
df[df['Country/Region']!='United States']

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,State,County,isSpecialLocation
0,not_provided,Thailand,15.0000,101.0000,2020-01-22,2,0,0,not_provided,not_provided,False
1,not_provided,Japan,36.0000,138.0000,2020-01-22,2,0,0,not_provided,not_provided,False
2,not_provided,Singapore,1.2833,103.8333,2020-01-22,0,0,0,not_provided,not_provided,False
3,not_provided,Nepal,28.1667,84.2500,2020-01-22,0,0,0,not_provided,not_provided,False
4,not_provided,Malaysia,2.5000,112.5000,2020-01-22,0,0,0,not_provided,not_provided,False
5,British Columbia,Canada,49.2827,-123.1207,2020-01-22,0,0,0,British Columbia,not_provided,False
6,New South Wales,Australia,-33.8688,151.2093,2020-01-22,0,0,0,New South Wales,not_provided,False
7,Victoria,Australia,-37.8136,144.9631,2020-01-22,0,0,0,Victoria,not_provided,False
8,Queensland,Australia,-28.0167,153.4000,2020-01-22,0,0,0,Queensland,not_provided,False
9,not_provided,Cambodia,11.5500,104.9167,2020-01-22,0,0,0,not_provided,not_provided,False
